In [2]:
pip install langchain-community langchain-openai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: lang

In [18]:
!pip install pydantic==2.10.1 chainlit

In [4]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 28.1 MB/s eta 0:00:00


In [5]:
#importing all the necessaries libraries

import os
from dotenv import load_dotenv
from typing import List
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chainlit as cl
import json

In [12]:
FEEDBACK_FILE = "feedback_log.json"
api_key = "your_api_key"

In [7]:
# Function to load and process PDFs from a folder
def process_pdf_folder(folder_path: str) -> List[Document]:
    documents = []
    for file in os.listdir(folder_path):
        if file.endswith('.pdf'):
            file_path = os.path.join(folder_path, file)
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return text_splitter.split_documents(documents)

In [13]:
# Function to initialize and save the vector store
def setup_vector_store(chunks: List[Document], store_path: str):
    vector_store = FAISS.from_documents(documents=chunks, embedding=OpenAIEmbeddings(api_key=api_key))
    vector_store.save_local(store_path)
    return vector_store

In [14]:
# Function to save feedback

def save_feedback(query: str, response: str, feedback: str):
    feedback_data = {"query": query, "response": response, "feedback": feedback}
    if os.path.exists(FEEDBACK_FILE):
        with open(FEEDBACK_FILE, "r") as file:
            feedback_list = json.load(file)
    else:
        feedback_list = []
    feedback_list.append(feedback_data)
    with open(FEEDBACK_FILE, "w") as file:
        json.dump(feedback_list, file, indent=4)

In [15]:
# Load and process PDF documents
pdf_folder = "./relevant_docs"
processed_chunks = process_pdf_folder(pdf_folder)

# Set up and save vector store
vector_store_path = "./vector_store"
vector_store = setup_vector_store(processed_chunks, vector_store_path)

# Initialize the chatbot model
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key=api_key)

In [20]:
# Chatbot behavior
@cl.on_chat_start
async def on_chat_start():
    retriever = vector_store.as_retriever()
    message_history = ChatMessageHistory()
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        output_key="response",
        chat_memory=message_history,
        return_messages=True,
    )

    chat_chain = ConversationalRetrievalChain.from_llm(
        llm=chat_model,
        chain_type="stuff",
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
    )
    cl.user_session.set("chat_chain", chat_chain)

@cl.on_message
async def handle_message(message: cl.Message):
    chat_chain = cl.user_session.get("chat_chain")
    response = await chat_chain.acall(message.content)
    answer = response["response"]
    source_docs = response.get("source_documents", [])

    sources = "\nSources: " + ", ".join([f"Page {doc.metadata['page_number']}" for doc in source_docs])
    await cl.Message(content=answer + sources).send()

    # 4. Asking for feedback
    feedback_message = "Was this answer helpful? Please reply with 'Yes' or 'No'."
    await cl.Message(content=feedback_message).send()

    # 5. Handling the feedback
    user_feedback = await cl.next_message()
    feedback = user_feedback.content.lower()
    save_feedback(query, response, feedback)
    feedback_reply = "Thank you for your feedback!" if feedback in ["yes", "no"] else "Invalid feedback received. Please reply with 'Yes' or 'No'."
    await cl.Message(content=feedback_reply).send()

